In [ ]:
import typing

import numpy as np
import pandas as pd
import requests

In [ ]:
DATA_PATH: str = './dataset'
DATA_SUBSET: str = 'full'

LANGUAGE: str = 'German'
TOPIC: str = 'ukraine'
SAMPLE_SIZE: int = 250

MODEL: str = 'llama3:70b-instruct-q6_K' # 'llama3:70b-instruct-q6_K', 'mixtral:8x7b-instruct-v0.1-q6_K'

In [ ]:
SYSTEM: str = \
    """
	Retrieve arguments from the following list of social media contents and list them in your response. Adhere to the structure of the examples below.
	
	Posts:
	[
        "Sad to see Ukraine's energy infrastructure under attack while the West twiddles its thumbs on providing critical military aid - time to put America first and focus on our own national security!", 
        "It's unconscionable that the US House of Representatives is dragging their feet on passing a Ukraine aid bill. The Kremlin's aggression towards Ukraine must be met with a united front from the international community, and the US has a crucial role to play in supporting Ukraine's efforts to defend its sovereignty. Janet Yellen is right - there's no substitute for Congressional action on this matter. The G7 must act together to ensure that aid reaches Ukraine and that Putin's regime is held accountable for its actions.", 
        "📢 US Treasury Sec Yellen urges Congress to pass Ukraine aid bill, stating there's 'no substitute' for our support in this critical time. Let's stand together with Ukraine, managing risks as a united G7 to uphold human rights, peace, and democracy worldwide! 🌍🇺🇦", "Standing with Ukraine 🇺🇦, we must urgently pass the aid bill. The time for debate is over; it's time for action. Our unity & support are crucial for justice and peace. #StandWithUkraine 🌍💪"
	]
	
	Arguments:
	[
        "Ukraine's energy infrastructure is under attack", 
        "The West, particularly the US, should provide critical military aid to Ukraine", 
        "The US should prioritize its national security and take action in supporting Ukraine", 
        "The US House of Representatives is delaying the passage of a Ukraine aid bill", 
        "The Kremlin's aggression towards Ukraine must be met with a united front from the international community", 
        "The US has a crucial role to play in supporting Ukraine's efforts to defend its sovereignty", 
        "There is no substitute for Congressional action on this matter", 
        "US Treasury Secretary Janet Yellen urges Congress to pass the Ukraine aid bill", 
        "The G7 must act together to ensure that aid reaches Ukraine and that Putin's regime is held accountable for its actions", 
        "It's time for the US to stand together with Ukraine, managing risks as a united G7 to uphold human rights, peace, and democracy worldwide", 
        "The aid bill must be passed urgently", "The time for debate is over; it's time for action", 
        "Unity and support from the US and the international community are crucial for justice and peace in Ukraine"
	]
	"""

In [ ]:
data: pd.DataFrame = (
    pd.read_parquet(f'{DATA_PATH}.{DATA_SUBSET}.parquet')
    .pipe(lambda _df: _df[_df['language'] == LANGUAGE])
    .pipe(lambda _df: _df[_df['topic'] == TOPIC])
)
data

In [ ]:
data_chunks: typing.List[pd.DataFrame] = np.array_split(data, len(data) / SAMPLE_SIZE)
len(data_chunks)

In [ ]:
for idx, chunk in enumerate(data_chunks):
    print(
        requests.post(
            'https://inf.cl.uni-trier.de/',
            json={
                'model': MODEL,
                'system': SYSTEM,
                'prompt': f'Posts:\n{chunk["text"].tolist()}\n\nArguments:\n'
            }
        ).json()['response']
    )
    break
